In [3]:
!pip install pdfplumber pandas openpyxl xlsxwriter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 62.1 MB/s eta 0:00:00


# New Section

# New Section

In [21]:
import pdfplumber
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment, PatternFill

# Define file paths (update these paths to match your local file locations)
pdf_files = {
    "cardio_structured": "cardio_structured.pdf",
    "prot_sap_102": "prot_sap_102.pdf",
    "prot_sap_1": "prot_sap_1.pdf"
}

# Pages to extract tables from
pages_to_extract = {
    "cardio_structured": 6,
    "prot_sap_102": 50,
    "prot_sap_1": 14
}

# Create workbook and worksheet
output_file = "Combined_Trial_Data.xlsx"
wb = Workbook()

# Remove default sheet if exists
if "Sheet" in wb.sheetnames:
    del wb["Sheet"]

# ================== Cardio Structured Table Creation ==================
ws_cardio = wb.create_sheet(title="cardio_structured", index=0)

# Add data with proper superscripts
data = [
    ["Visit", "X³", "EOT⁴", "FU⁵"],
    ["Week", "every 16w", "", "EOT +28 days"],
    ["Day Time window (days)", "±14", "", "+7"],
    ["Informed consent", "X²", "", ""],
    ["IVRS/IWRS", "X", "X", ""],
    ["Dispense trial drug", "X", "", ""],
    ["Collect used and unused trial drug", "X", "X", ""],
    ["Adverse events", "X", "X", "X"],
    ["Concomitant therapy", "X", "X", "X"],
    ["Trial medication termination", "", "X", ""],
    ["Conclusion of patient participation", "", "", "X"]
]

# Write data to worksheet
for row_idx, row in enumerate(data, 1):
    for col_idx, value in enumerate(row, 1):
        cell = ws_cardio.cell(row=row_idx, column=col_idx, value=value)

# Create styles for Cardio Structured sheet
header_fill = PatternFill(start_color="D3D3D3", fill_type="solid")
thin_border = Border(left=Side(style='thin'),
                    right=Side(style='thin'),
                    top=Side(style='thin'),
                    bottom=Side(style='thin'))
center_alignment = Alignment(horizontal='center', vertical='center')

# Apply formatting to Cardio Structured sheet
for row in ws_cardio.iter_rows():
    for cell in row:
        cell.border = thin_border
        cell.alignment = center_alignment

        # Header row styling
        if cell.row == 1:
            cell.fill = header_fill
            cell.font = Font(name='Calibri', size=11, bold=True)
        elif cell.row <= 3:
            cell.font = Font(name='Calibri', size=11, bold=True)
        else:
            cell.font = Font(name='Calibri', size=11)

# Set column widths and row heights for Cardio Structured
ws_cardio.column_dimensions['A'].width = 40
ws_cardio.column_dimensions['B'].width = 12
ws_cardio.column_dimensions['C'].width = 12
ws_cardio.column_dimensions['D'].width = 20

row_heights = {1: 25, 2: 20, 5: 20, 8: 20, 12: 20}
for row, height in row_heights.items():
    ws_cardio.row_dimensions[row].height = height

# ================== PDF Table Extraction ==================
def extract_table(pdf_path, page_number):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number - 1]
        table = page.extract_table()
        return pd.DataFrame(table[1:], columns=table[0]) if table else None

def get_border_style(pdf_name):
    if pdf_name == "cardio_structured":
        return Border(left=Side(style="thick"), right=Side(style="thick"),
                      top=Side(style="thick"), bottom=Side(style="thick"))
    elif pdf_name == "prot_sap_102":
        return Border(left=Side(style="thin"), right=Side(style="thin"),
                      top=Side(style="thin"), bottom=Side(style="thin"))
    else:
        return Border(left=Side(style="thin"), right=Side(style="thin"),
                      top=Side(style="thin"), bottom=Side(style="thin"))

def format_pdf_sheet(ws, pdf_name):
    header_font = Font(name="Times New Roman", size=12, bold=False)  # Headers
    content_font = Font(name="Calibri", size=11, bold=False)         # Content
    bold_font = Font(name="Calibri", size=11, bold=True)             # Bold content
    alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)

    # Get the appropriate border style
    border = get_border_style(pdf_name)

    # Auto-adjust column width and apply styles
    for col in ws.columns:
        max_length = 0
        col_letter = col[0].column_letter
        for cell in col:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))

            # Align first column content to the left and others to the center
            if cell.column == 1:
                cell.alignment = Alignment(horizontal="left", vertical="center", wrap_text=True)
            else:
                cell.alignment = alignment

            # Apply borders
            cell.border = border

            # Bold the first two rows for `prot_sap_102`
            if pdf_name == "prot_sap_102" and (cell.row == 2 or (cell.row==1 and cell.column==1)) :
                cell.font = bold_font

            else:
                # Format headers in Times New Roman, content in Calibri
                if cell.row == 1 or cell.column == 1:
                    cell.font = header_font
                else:
                    cell.font = content_font


        # Dynamically set the column width based on max_length

        col_width = max_length * 0.8
        if pdf_name=="prot_sap_102":
          col_width = min(max(col_width, 8), 20)
        elif pdf_name=="prot_sap_1":
          col_width = min(max(col_width,20),15)
        ws.column_dimensions[col_letter].width = col_width
# Extract tables from PDFs and add to workbook
for sheet_name, pdf_path in pdf_files.items():
    # Skip processing if sheet already exists (like "cardio_structured")
    if sheet_name in wb.sheetnames or sheet_name.replace("_", " ").title() in wb.sheetnames:
        continue

    page_num = pages_to_extract[sheet_name]
    print(f"📄 Extracting from {sheet_name} (Page {page_num})...")

    df = extract_table(pdf_path, page_num)

    if df is not None:
        ws = wb.create_sheet(title=sheet_name.replace("_", " ").title())
        for r_idx, row in enumerate([df.columns] + df.values.tolist(), start=1):
            for c_idx, value in enumerate(row, start=1):
                ws.cell(row=r_idx, column=c_idx, value=value)
        format_pdf_sheet(ws, sheet_name)
        print(" Table extracted and formatted successfully!")
    else:
        print(" No table detected. OCR might be needed.")

# Save the final combined file
wb.save(output_file)
print(f"✅ Combined file saved to {output_file}!")


📄 Extracting from prot_sap_102 (Page 50)...
 Table extracted and formatted successfully!
📄 Extracting from prot_sap_1 (Page 14)...
 Table extracted and formatted successfully!
✅ Combined file saved to Combined_Trial_Data.xlsx!
